In [137]:
import numpy as np
from numpy.linalg import det
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d, Delaunay
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [552]:


points = np.array([[0,0],[0,1.1],[1,0],[1,1]])
tri1 = Delaunay(points)
print(points[tri1.simplices][1])
plt.triplot(points[:,0], points[:,1], tri1.simplices)

plt.plot(points[:,0], points[:,1], 'o')

plt.show()

[[1.  1. ]
 [0.  1.1]
 [0.  0. ]]


In [553]:
# generate a set of 3D points
points = np.random.rand(10, 3)

# compute the Delaunay triangulation
tri = Delaunay(points)

# visualize the result

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(points[:,0], points[:,1], points[:,2], triangles=tri.simplices)
plt.show()

[[0.09774871 0.5470768  0.15891889]
 [0.11901355 0.11310006 0.91102531]
 [0.59808732 0.25015872 0.07144888]
 [0.53618142 0.14480297 0.77840293]
 [0.49610966 0.72644888 0.3957266 ]
 [0.70232274 0.68461436 0.56141621]
 [0.84573963 0.58247357 0.57811041]
 [0.30798311 0.93150627 0.51713957]
 [0.39171466 0.54164229 0.14524214]
 [0.26512524 0.30668186 0.51515877]]


# Code AI

In [535]:
def delaunay_2D(points):
    """
    This function takes a list of points as input and returns the Delaunay triangulation of the points.
    It does not use the scipy library.
    
    Parameters:
    points (list): A list of tuples representing the points in 2D space. Each tuple should have two elements.
    
    Returns:
    list: A list of tuples representing the edges of the Delaunay triangulation. Each tuple should have two elements.
    """
    try:
        # Check if the input is a list
        if not isinstance(points, list):
            raise TypeError("Input must be a list")
        
        # Check if the list contains tuples with two elements
        for point in points:
            if not isinstance(point, tuple) or len(point) != 2:
                raise ValueError("Each point must be a tuple with two elements")
        
        # Perform Delaunay triangulation
        edges = []
        for i in range(len(points)):
            for j in range(i+1, len(points)):
                for k in range(j+1, len(points)):
                    # Check if the three points form a circle that contains no other points
                    x1, y1 = points[i]
                    x2, y2 = points[j]
                    x3, y3 = points[k]
                    a = x2 - x1
                    b = y2 - y1
                    c = x3 - x1
                    d = y3 - y1
                    e = a*(x1 + x2) + b*(y1 + y2)
                    f = c*(x1 + x3) + d*(y1 + y3)
                    g = 2*(a*(y3 - y2) - b*(x3 - x2))
                    if g == 0:
                        continue
                    center_x = (d*e - b*f) / g
                    center_y = (a*f - c*e) / g
                    radius = ((x1 - center_x)**2 + (y1 - center_y)**2)**0.5
                    contains_point = False
                    for p in points:
                        if p == points[i] or p == points[j] or p == points[k]:
                            continue
                        if ((p[0] - center_x)**2 + (p[1] - center_y)**2)**0.5 < radius:
                            contains_point = True
                            break
                    if not contains_point:
                        edges.append((i, j))
                        edges.append((j, k))
                        edges.append((k, i))
        
        # Remove duplicate edges
        edges = list(set(edges))
        
        return edges
    except Exception as e:
        # Log the error
        print(f"Error: {e}")
        return []
    

In [527]:
def delaunay_3D(points):
    """
    This function takes a list of 3D points and returns a list of tetrahedrons that form the Delaunay triangulation.
    
    Parameters:
    points (list): A list of 3D points in the form [(x1, y1, z1), (x2, y2, z2), ...]
    
    Returns:
    list: A list of tetrahedrons in the form [(p1, p2, p3, p4), (p1, p2, p3, p4), ...]
    """
    try:
        # Check if the input is a list
        if not isinstance(points, list):
            raise TypeError("Input must be a list of 3D points")
        
        # Check if each point is a tuple of length 3
        for point in points:
            if not isinstance(point, tuple) or len(point) != 3:
                raise TypeError("Each point must be a tuple of length 3")
        
        # Create a list of tetrahedrons
        tetrahedrons = []
        for i in range(len(points)):
            for j in range(i+1, len(points)):
                for k in range(j+1, len(points)):
                    for l in range(k+1, len(points)):
                        
                        # Check if the four points form a circumsphere that contains no other points
                        circumsphere = _circumsphere(points[i], points[j], points[k], points[l])
                        if all(_point_in_sphere(point, circumsphere) for point in points if point not in (points[i], points[j], points[k], points[l])):
                            tetrahedrons.append((points[i], points[j], points[k], points[l]))
        
        return tetrahedrons
    except TypeError as e:
        # Log the error
        print(f"Error: {e}")
        return []
    
def _circumsphere(p1, p2, p3, p4):
    """
    This function takes four 3D points as arguments and returns the circumsphere that passes through them.
    
    Parameters:
    p1 (tuple): The first 3D point as a tuple (x, y, z)
    p2 (tuple): The second 3D point as a tuple (x, y, z)
    p3 (tuple): The third 3D point as a tuple (x, y, z)
    p4 (tuple): The fourth 3D point as a tuple (x, y, z)
    
    Returns:
    tuple: The center and radius of the circumsphere as a tuple (x, y, z, r)
    """
    try:
        # Check if all arguments are tuples of length 3
        if not all(isinstance(p, tuple) and len(p) == 3 for p in [p1, p2, p3, p4]):
            raise TypeError("All arguments must be tuples of length 3")
        
        # Calculate the coefficients of the matrix
        a = p2[0] - p1[0]
        b = p2[1] - p1[1]
        c = p2[2] - p1[2]
        d = p3[0] - p1[0]
        e = p3[1] - p1[1]
        f = p3[2] - p1[2]
        g = p4[0] - p1[0]
        h = p4[1] - p1[1]
        i = p4[2] - p1[2]
        D = 2 * (a * (e * i - h * f) - b * (d * i - g * f) + c * (d * h - g * e))
        
        # Calculate the center of the circumsphere
        x = ((e * i - h * f) * (a**2 + b**2 + c**2) - (d * i - g * f) * (b**2 + c**2 + (p3[1] - p1[1])**2) + (d * h - g * e) * (c**2 + (p3[2] - p1[2])**2 + (p3[0] - p1[0])**2)) / D
        y = ((d * i - g * f) * (a**2 + b**2 + c**2) - (e * i - h * f) * (a**2 + c**2 + (p2[0] - p1[0])**2) + (d * h - g * e) * ((p3[1] - p1[1])**2 + (p3[0] - p1[0])**2 + c**2)) / D
        z = ((d * h - g * e) * (a**2 + b**2 + c**2) - (e * i - h * f) * (b**2 + c**2 + (p2[1] - p1[1])**2) + (d * i - g * f) * ((p3[2] - p1[2])**2 + (p3[0] - p1[0])**2 + b**2)) / D
        
        # Calculate the radius of the circumsphere
        r = math.sqrt((x - p1[0])**2 + (y - p1[1])**2 + (z - p1[2])**2)
        
        # Return the center and radius of the circumsphere
        return (x, y, z, r)
    except TypeError as e:
        # Log the error
        print(f"Error: {e}")
        return None


def _point_in_sphere(point, sphere):
    """
    This function takes a 3D point and a circumsphere and returns True if the point is inside the sphere.
    
    Parameters:
    point (tuple): The 3D point
    sphere (tuple): The center and radius of the circumsphere in the form (center, radius)
    
    Returns:
    bool: True if the point is inside the sphere, False otherwise
    """
    return math.sqrt((point[0]-sphere[0])**2 + (point[1]-sphere[1])**2 + (point[2]-sphere[2])**2) < sphere[1]


# Testing Area

In [558]:
def get_first_value(tuples_list):
    """
    This function takes a list of tuples as argument and returns a list containing the first value of each tuple.
    
    Parameters:
    tuples_list (list): A list of tuples
    
    Returns:
    list: A list containing the first value of each tuple
    """
    try:
        # Check if the argument is a list
        if not isinstance(tuples_list, list):
            raise TypeError("The argument must be a list")
        
        # Extract the first value of each tuple and return as a list
        return [t[0] for t in tuples_list]
    except TypeError as e:
        # Log the error
        print(f"Error: {e}")
        return []

def get_second_value(tuples_list):
    """
    This function takes a list of tuples as argument and returns a list containing the first value of each tuple.
    
    Parameters:
    tuples_list (list): A list of tuples
    
    Returns:
    list: A list containing the first value of each tuple
    """
    try:
        # Check if the argument is a list
        if not isinstance(tuples_list, list):
            raise TypeError("The argument must be a list")
        
        # Extract the first value of each tuple and return as a list
        return [t[1] for t in tuples_list]
    except TypeError as e:
        # Log the error
        print(f"Error: {e}")
        return []    

def get_third_value(tuples_list):
    """
    This function takes a list of tuples as argument and returns a list containing the first value of each tuple.
    
    Parameters:
    tuples_list (list): A list of tuples
    
    Returns:
    list: A list containing the first value of each tuple
    """
    try:
        # Check if the argument is a list
        if not isinstance(tuples_list, list):
            raise TypeError("The argument must be a list")
        
        # Extract the first value of each tuple and return as a list
        return [t[2] for t in tuples_list]
    except TypeError as e:
        # Log the error
        print(f"Error: {e}")
        return [] 
%matplotlib

################################################################################
# points = np.random.rand(25,2)

################################################################################
# Define the function that describes the surface
def f(x, y):
    return np.cos(np.sqrt(x**2 + y**2))

# Define the x, y coordinates of the points on the surface
x = np.linspace(1, 15, 6)
y = np.linspace(1, 15, 6)

# Create a meshgrid from the x, y coordinates
X, Y = np.meshgrid(x, y)

# Calculate the corresponding z coordinates using the surface function
Z = f(X, Y)

# Combine the x, y, and z coordinates into a 3D array of points
points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
################################################################################
# # Define the side length of the cube
# side_length = 5.0

# # Define the number of points to generate
# num_points = 10

# # Generate random points within the cube
# x = np.random.uniform(-side_length/2, side_length/2, num_points)
# y = np.random.uniform(-side_length/2, side_length/2, num_points)
# z = np.random.uniform(-side_length/2, side_length/2, num_points)

# # Combine the x, y, and z coordinates into a 3D array of points
# points = np.vstack([x.ravel(), y.ravel(), z.ravel()]).T
################################################################################
D = 3

points.tolist()
points1 = []
for i in range(len(points)):
    if D == 2:
        points1.append((points[i,0],points[i,1]))
        tri = delaunay_2D(points1) 
    else:
        points1.append((points[i,0],points[i,1],points[i,2]))
        tri = delaunay_3D(points1) 


################################################################################
# 3D
index = [sp for tpl in tri for sp in tpl]
print(points)
print(index)
x_index = get_first_value(index)
y_index = get_second_value(index)
z_index = get_third_value(index)


# Plot the results (optional)
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(x_index, y_index, z_index)


################################################################################
# # 2D
# index = get_first_value(tri)

# plt.triplot(points[index][:,0], points[index][:,1])

# plt.show()


Using matplotlib backend: MacOSX


/var/folders/1p/cp6mx14s7fbcxp3s3vtxppl00000gn/T/ipykernel_11698/1237089175.py:70: RuntimeWarning: divide by zero encountered in double_scalars
  x = ((e * i - h * f) * (a**2 + b**2 + c**2) - (d * i - g * f) * (b**2 + c**2 + (p3[1] - p1[1])**2) + (d * h - g * e) * (c**2 + (p3[2] - p1[2])**2 + (p3[0] - p1[0])**2)) / D
/var/folders/1p/cp6mx14s7fbcxp3s3vtxppl00000gn/T/ipykernel_11698/1237089175.py:71: RuntimeWarning: divide by zero encountered in double_scalars
  y = ((d * i - g * f) * (a**2 + b**2 + c**2) - (e * i - h * f) * (a**2 + c**2 + (p2[0] - p1[0])**2) + (d * h - g * e) * ((p3[1] - p1[1])**2 + (p3[0] - p1[0])**2 + c**2)) / D
/var/folders/1p/cp6mx14s7fbcxp3s3vtxppl00000gn/T/ipykernel_11698/1237089175.py:72: RuntimeWarning: divide by zero encountered in double_scalars
  z = ((d * h - g * e) * (a**2 + b**2 + c**2) - (e * i - h * f) * (b**2 + c**2 + (p2[1] - p1[1])**2) + (d * i - g * f) * ((p3[2] - p1[2])**2 + (p3[0] - p1[0])**2 + b**2)) / D
/var/folders/1p/cp6mx14s7fbcxp3s3vtxppl0000

[[ 1.          1.          0.15594369]
 [ 3.8         1.         -0.70541781]
 [ 6.6         1.          0.92409242]
 [ 9.4         1.         -0.9996006 ]
 [12.2         1.          0.94750515]
 [15.          1.         -0.78091507]
 [ 1.          3.8        -0.70541781]
 [ 3.8         3.8         0.61439785]
 [ 6.6         3.8         0.23596212]
 [ 9.4         3.8        -0.75558112]
 [12.2         3.8         0.97766763]
 [15.          3.8        -0.97272017]
 [ 1.          6.6         0.92409242]
 [ 3.8         6.6         0.23596212]
 [ 6.6         6.6        -0.99586522]
 [ 9.4         6.6         0.4706867 ]
 [12.2         6.6         0.26319669]
 [15.          6.6        -0.77767516]
 [ 1.          9.4        -0.9996006 ]
 [ 3.8         9.4        -0.75558112]
 [ 6.6         9.4         0.4706867 ]
 [ 9.4         9.4         0.7470142 ]
 [12.2         9.4        -0.95334576]
 [15.          9.4         0.41069638]
 [ 1.         12.2         0.94750515]
 [ 3.8        12.2       

In [562]:
def delaunay_triangulation_dv(points):
    """
    This function performs Delaunay triangulation using divide and conquer algorithm without using scipy.spatial.
    
    Parameters:
    points (list): A list of tuples representing the points to be triangulated. Each tuple should contain the x and y coordinates of the point.
    
    Returns:
    list: A list of tuples representing the edges of the Delaunay triangulation. Each tuple contains the indices of the two points that form the edge.
    """
    try:
        # Check if the input is a list of tuples
        if not isinstance(points, list) or not all(isinstance(point, tuple) and len(point) == 2 for point in points):
            raise TypeError("Input must be a list of tuples with two elements each")
        
        # Sort the points by x-coordinate
        points = sorted(points, key=lambda point: point[0])
        
        # Define the recursive function for divide and conquer
        def divide_and_conquer(l, r):
            if r - l == 1:
                return []
            elif r - l == 2:
                return [(l, r-1)]
            else:
                m = (l + r) // 2
                left_edges = divide_and_conquer(l, m)
                right_edges = divide_and_conquer(m, r)
                return merge(left_edges, right_edges, points, l, r)
        
        # Define the function for merging two sets of edges
        def merge(left_edges, right_edges, points, l, r):
            left_hull = get_hull(left_edges, points, l, r)
            right_hull = get_hull(right_edges, points, l, r)
            left_index = find_tangent(left_hull, right_hull, points)
            right_index = find_tangent(right_hull, left_hull, points, reverse=True)
            result = left_edges + right_edges
            
            # Add the new edges formed by the two tangent points
            result.append((left_index, right_index))
            i, j = left_index, right_index
            while i != left_hull[-1] or j != right_hull[-1]:
                if (points[j][0] - points[i][0]) * (points[left_hull[(left_hull.index(i) + 1) % len(left_hull)][0]][1] - points[i][1]) > (points[left_hull[(left_hull.index(i) + 1) % len(left_hull)][0]][0] - points[i][0]) * (points[j][1] - points[i][1]):
                    i = left_hull[(left_hull.index(i) + 1) % len(left_hull)][0]
                else:
                    j = right_hull[(right_hull.index(j) + 1) % len(right_hull)][0]
                result.append((i, j))
            
            return result
        
        # Define the function for getting the convex hull of a set of edges
        def get_hull(edges, points, l, r):
            hull = []
            for edge in edges:
                if edge[0] < l or edge[0] >= r or edge[1] < l or edge[1] >= r:
                    continue
                if not hull:
                    hull = list(edge)
                elif points[edge[1]][1] > points[hull[-1]][1]:
                    hull.append(edge[1])
                else:
                    while len(hull) > 1 and (points[edge[1]][1] - points[hull[-2]][1]) * (points[hull[-1]][0] - points[hull[-2]][0]) < (points[hull[-1]][1] - points[hull[-2]][1]) * (points[edge[1]][0] - points[hull[-2]][0]):
                        hull.pop()
                    hull.append(edge[1])
            return hull
        
        # Define the function for finding the tangent point between two convex hulls
        def find_tangent(left_hull, right_hull, points, reverse=False):
            i, j = 0, 0
            while True:
                if (points[left_hull[i]][0] < points[right_hull[j]][0]) ^ reverse:
                    i += 1
                else:
                    j += 1
                if i == len(left_hull) or j == len(right_hull):
                    break
            return left_hull[i-1] if i > 0 else left_hull[-1]
        
        # Perform the divide and conquer algorithm
        return divide_and_conquer(0, len(points))
    except TypeError as e:
        # Log the error
        print(f"Error: {e}")
        return []


In [564]:
D = 2
points = np.random.rand(25,2)
points.tolist()
points1 = []
for i in range(len(points)):
    if D == 2:
        points1.append((points[i,0],points[i,1]))
        tri = delaunay_triangulation_dv(points1) 
    else:
        points1.append((points[i,0],points[i,1],points[i,2]))
        tri = delaunay_3D(points1) 


################################################################################
# 3D
# index = [sp for tpl in tri for sp in tpl]
# print(points)
# print(index)
# x_index = get_first_value(index)
# y_index = get_second_value(index)
# z_index = get_third_value(index)


# # Plot the results (optional)
# plt.close('all')
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.plot_trisurf(x_index, y_index, z_index)


################################################################################
# 2D
index = get_first_value(tri)

plt.triplot(points[index][:,0], points[index][:,1])

plt.show()

IndexError: list index out of range